# Lesson I6: Traits & Shared Behavior

**Duration**: 120-135 minutes  
**Stage**: Intermediate (Building Skills)

---

## 🎯 Learning Objectives

By the end of this lesson, you will be able to:
1. Define traits to specify shared behavior across types
2. Implement traits for custom and existing types
3. Use trait bounds to constrain generic parameters
4. Work with default implementations and associated types
5. Understand the orphan rule and trait coherence

---

## 📋 Prerequisite Review

**Quick Check**: From our previous lessons:

1. How do you create a generic function with type constraints?
2. What is monomorphization?
3. How do you specify multiple type parameters?
4. What's the purpose of the `where` clause?

**Answers**: 1) `fn func<T: Constraint>(param: T)`, 2) Compiler generates specific versions for each type, 3) `<T, U, V>`, 4) Alternative syntax for complex trait bounds

---

## 🧠 Key Concepts

### What are Traits?

**Traits** define shared behavior:
- **Interface**: Specify what methods a type must implement
- **Polymorphism**: Different types can implement the same trait
- **Constraints**: Used as bounds on generic parameters
- **Composition**: Types can implement multiple traits

### Trait Implementation Rules

- **Orphan Rule**: Can implement trait for type if you own either the trait or the type
- **Coherence**: No overlapping implementations allowed
- **Default Methods**: Traits can provide default implementations
- **Associated Types**: Traits can have associated types and constants

---

## 🔬 Live Code Exploration

### Basic Trait Definition and Implementation

In [ ]:
// Basic trait definition and implementation

trait Drawable {
    fn draw(&self);
    fn area(&self) -> f64;
    
    // Default implementation
    fn describe(&self) {
        println!("This shape has an area of {:.2}", self.area());
    }
}

struct Circle {
    radius: f64,
}

impl Circle {
    fn new(radius: f64) -> Self {
        Circle { radius }
    }
}

impl Drawable for Circle {
    fn draw(&self) {
        println!("Drawing a circle with radius {}", self.radius);
    }
    
    fn area(&self) -> f64 {
        std::f64::consts::PI * self.radius * self.radius
    }
}

struct Rectangle {
    width: f64,
    height: f64,
}

impl Rectangle {
    fn new(width: f64, height: f64) -> Self {
        Rectangle { width, height }
    }
}

impl Drawable for Rectangle {
    fn draw(&self) {
        println!("Drawing a rectangle {}x{}", self.width, self.height);
    }
    
    fn area(&self) -> f64 {
        self.width * self.height
    }
    
    // Override default implementation
    fn describe(&self) {
        println!("Rectangle: {}x{} with area {:.2}", 
                self.width, self.height, self.area());
    }
}

// Function that works with any Drawable
fn draw_shape(shape: &dyn Drawable) {
    shape.draw();
    shape.describe();
}

// Generic function with trait bound
fn print_area<T: Drawable>(shape: &T) {
    println!("Area: {:.2}", shape.area());
}

fn basic_traits_demo() {
    println!("=== Basic Traits Demo ===");
    
    let circle = Circle::new(5.0);
    let rectangle = Rectangle::new(4.0, 6.0);
    
    // Direct method calls
    circle.draw();
    circle.describe();
    
    rectangle.draw();
    rectangle.describe();
    
    println!();
    
    // Using trait objects
    let shapes: Vec<&dyn Drawable> = vec![&circle, &rectangle];
    
    for (i, shape) in shapes.iter().enumerate() {
        println!("Shape {}:", i + 1);
        draw_shape(*shape);
        println!();
    }
    
    // Using generic function
    print_area(&circle);
    print_area(&rectangle);
}

basic_traits_demo();

### Advanced Trait Features

In [ ]:
// Advanced trait features: associated types, constants, and complex bounds

trait Iterator {
    type Item; // Associated type
    
    fn next(&mut self) -> Option<Self::Item>;
    
    // Default implementation using associated type
    fn collect<C: FromIterator<Self::Item>>(self) -> C
    where
        Self: Sized,
    {
        FromIterator::from_iter(self)
    }
}

trait FromIterator<T> {
    fn from_iter<I: Iterator<Item = T>>(iter: I) -> Self;
}

// Custom iterator implementation
struct Counter {
    current: usize,
    max: usize,
}

impl Counter {
    fn new(max: usize) -> Self {
        Counter { current: 0, max }
    }
}

impl Iterator for Counter {
    type Item = usize;
    
    fn next(&mut self) -> Option<Self::Item> {
        if self.current < self.max {
            let current = self.current;
            self.current += 1;
            Some(current)
        } else {
            None
        }
    }
}

// Trait with associated constants
trait Numeric {
    const ZERO: Self;
    const ONE: Self;
    
    fn add(self, other: Self) -> Self;
    fn multiply(self, other: Self) -> Self;
    
    fn square(self) -> Self
    where
        Self: Sized + Copy,
    {
        self.multiply(self)
    }
}

impl Numeric for i32 {
    const ZERO: Self = 0;
    const ONE: Self = 1;
    
    fn add(self, other: Self) -> Self {
        self + other
    }
    
    fn multiply(self, other: Self) -> Self {
        self * other
    }
}

impl Numeric for f64 {
    const ZERO: Self = 0.0;
    const ONE: Self = 1.0;
    
    fn add(self, other: Self) -> Self {
        self + other
    }
    
    fn multiply(self, other: Self) -> Self {
        self * other
    }
}

// Generic function with multiple trait bounds
fn sum_of_squares<T>(values: &[T]) -> T
where
    T: Numeric + Copy + std::fmt::Display,
{
    let mut sum = T::ZERO;
    for &value in values {
        let squared = value.square();
        println!("{}² = {}", value, squared);
        sum = sum.add(squared);
    }
    sum
}

// Trait with Self type
trait Cloneable {
    fn clone_self(&self) -> Self;
}

#[derive(Debug)]
struct Point {
    x: i32,
    y: i32,
}

impl Cloneable for Point {
    fn clone_self(&self) -> Self {
        Point { x: self.x, y: self.y }
    }
}

fn advanced_traits_demo() {
    println!("\n=== Advanced Traits Demo ===");
    
    // Custom iterator
    let mut counter = Counter::new(5);
    println!("Counter iterator:");
    while let Some(value) = counter.next() {
        println!("  {}", value);
    }
    
    // Associated constants
    println!("\nNumeric constants:");
    println!("i32 ZERO: {}, ONE: {}", i32::ZERO, i32::ONE);
    println!("f64 ZERO: {}, ONE: {}", f64::ZERO, f64::ONE);
    
    // Generic function with multiple bounds
    let integers = [1, 2, 3, 4];
    let int_sum = sum_of_squares(&integers);
    println!("\nSum of squares (integers): {}", int_sum);
    
    let floats = [1.5, 2.5, 3.5];
    let float_sum = sum_of_squares(&floats);
    println!("\nSum of squares (floats): {:.2}", float_sum);
    
    // Self type in traits
    let point = Point { x: 10, y: 20 };
    let cloned_point = point.clone_self();
    println!("\nOriginal point: {:?}", point);
    println!("Cloned point: {:?}", cloned_point);
}

advanced_traits_demo();

### Trait Objects and Dynamic Dispatch

In [ ]:
// Trait objects and dynamic dispatch

trait Animal {
    fn name(&self) -> &str;
    fn make_sound(&self);
    
    fn introduce(&self) {
        println!("Hi, I'm {} and I go:", self.name());
        self.make_sound();
    }
}

struct Dog {
    name: String,
    breed: String,
}

impl Dog {
    fn new(name: String, breed: String) -> Self {
        Dog { name, breed }
    }
    
    fn breed(&self) -> &str {
        &self.breed
    }
}

impl Animal for Dog {
    fn name(&self) -> &str {
        &self.name
    }
    
    fn make_sound(&self) {
        println!("Woof! Woof!");
    }
    
    fn introduce(&self) {
        println!("Hi, I'm {}, a {} dog, and I go:", self.name, self.breed);
        self.make_sound();
    }
}

struct Cat {
    name: String,
    indoor: bool,
}

impl Cat {
    fn new(name: String, indoor: bool) -> Self {
        Cat { name, indoor }
    }
}

impl Animal for Cat {
    fn name(&self) -> &str {
        &self.name
    }
    
    fn make_sound(&self) {
        if self.indoor {
            println!("Meow meow (softly)");
        } else {
            println!("MEOW! HISS!");
        }
    }
}

struct Bird {
    name: String,
    species: String,
}

impl Bird {
    fn new(name: String, species: String) -> Self {
        Bird { name, species }
    }
}

impl Animal for Bird {
    fn name(&self) -> &str {
        &self.name
    }
    
    fn make_sound(&self) {
        match self.species.as_str() {
            "parrot" => println!("Squawk! Hello! Squawk!"),
            "crow" => println!("Caw! Caw!"),
            "canary" => println!("Tweet tweet!"),
            _ => println!("Chirp chirp!"),
        }
    }
}

// Function that works with any Animal trait object
fn animal_chorus(animals: &[Box<dyn Animal>]) {
    println!("🎵 Animal Chorus Time! 🎵");
    for (i, animal) in animals.iter().enumerate() {
        println!("\n{}. ", i + 1);
        animal.introduce();
    }
}

// Generic function (static dispatch)
fn introduce_animal<T: Animal>(animal: &T) {
    animal.introduce();
}

// Function returning trait object
fn create_random_animal(seed: u32) -> Box<dyn Animal> {
    match seed % 3 {
        0 => Box::new(Dog::new("Rex".to_string(), "Golden Retriever".to_string())),
        1 => Box::new(Cat::new("Whiskers".to_string(), true)),
        _ => Box::new(Bird::new("Polly".to_string(), "parrot".to_string())),
    }
}

fn trait_objects_demo() {
    println!("\n=== Trait Objects and Dynamic Dispatch ===");
    
    // Create different animals
    let dog = Dog::new("Buddy".to_string(), "Labrador".to_string());
    let cat = Cat::new("Mittens".to_string(), false);
    let bird = Bird::new("Tweety".to_string(), "canary".to_string());
    
    // Static dispatch (compile-time)
    println!("Static dispatch:");
    introduce_animal(&dog);
    introduce_animal(&cat);
    introduce_animal(&bird);
    
    // Dynamic dispatch (runtime)
    let animals: Vec<Box<dyn Animal>> = vec![
        Box::new(dog),
        Box::new(cat),
        Box::new(bird),
    ];
    
    println!("\nDynamic dispatch:");
    animal_chorus(&animals);
    
    // Creating animals at runtime
    println!("\n\nRandom animals:");
    for i in 0..5 {
        let animal = create_random_animal(i * 7 + 3);
        print!("Random animal {}: ", i + 1);
        animal.introduce();
        println!();
    }
}

trait_objects_demo();

---

## 🎯 Guided Practice

### Exercise 1: Media Player System

Create a media player system using traits for different media types.

In [ ]:
// TODO: Complete the media player system using traits

trait Playable {
    fn play(&self);
    fn pause(&self);
    fn stop(&self);
    fn duration(&self) -> u32; // in seconds
    fn title(&self) -> &str;
    
    // Default implementation
    fn info(&self) {
        let minutes = self.duration() / 60;
        let seconds = self.duration() % 60;
        println!("'{}' - Duration: {}:{:02}", self.title(), minutes, seconds);
    }
}

trait Seekable {
    fn seek_to(&self, position: u32);
    fn fast_forward(&self, seconds: u32) {
        println!("Fast forwarding {} seconds", seconds);
    }
    fn rewind(&self, seconds: u32) {
        println!("Rewinding {} seconds", seconds);
    }
}

#[derive(Debug)]
struct Song {
    title: String,
    artist: String,
    duration: u32,
    genre: String,
}

impl Song {
    fn new(title: String, artist: String, duration: u32, genre: String) -> Self {
        Song { title, artist, duration, genre }
    }
    
    fn artist(&self) -> &str {
        &self.artist
    }
    
    fn genre(&self) -> &str {
        &self.genre
    }
}

impl Playable for Song {
    fn play(&self) {
        println!("🎵 Playing song: '{}' by {}", self.title, self.artist);
    }
    
    fn pause(&self) {
        println!("⏸️  Paused: '{}'", self.title);
    }
    
    fn stop(&self) {
        println!("⏹️  Stopped: '{}'", self.title);
    }
    
    fn duration(&self) -> u32 {
        self.duration
    }
    
    fn title(&self) -> &str {
        &self.title
    }
    
    fn info(&self) {
        let minutes = self.duration() / 60;
        let seconds = self.duration() % 60;
        println!("🎵 '{}' by {} [{}] - Duration: {}:{:02}", 
                self.title(), self.artist(), self.genre(), minutes, seconds);
    }
}

impl Seekable for Song {
    fn seek_to(&self, position: u32) {
        let minutes = position / 60;
        let seconds = position % 60;
        println!("🎵 Seeking '{}' to {}:{:02}", self.title, minutes, seconds);
    }
}

#[derive(Debug)]
struct Podcast {
    title: String,
    host: String,
    episode: u32,
    duration: u32,
}

impl Podcast {
    fn new(title: String, host: String, episode: u32, duration: u32) -> Self {
        Podcast { title, host, episode, duration }
    }
    
    fn host(&self) -> &str {
        &self.host
    }
    
    fn episode(&self) -> u32 {
        self.episode
    }
}

impl Playable for Podcast {
    fn play(&self) {
        println!("🎙️  Playing podcast: '{}' Episode {} with {}", 
                self.title, self.episode, self.host);
    }
    
    fn pause(&self) {
        println!("⏸️  Paused podcast: '{}' Episode {}", self.title, self.episode);
    }
    
    fn stop(&self) {
        println!("⏹️  Stopped podcast: '{}' Episode {}", self.title, self.episode);
    }
    
    fn duration(&self) -> u32 {
        self.duration
    }
    
    fn title(&self) -> &str {
        &self.title
    }
    
    fn info(&self) {
        let minutes = self.duration() / 60;
        let seconds = self.duration() % 60;
        println!("🎙️  '{}' Episode {} with {} - Duration: {}:{:02}", 
                self.title(), self.episode(), self.host(), minutes, seconds);
    }
}

impl Seekable for Podcast {
    fn seek_to(&self, position: u32) {
        let minutes = position / 60;
        let seconds = position % 60;
        println!("🎙️  Seeking '{}' Episode {} to {}:{:02}", 
                self.title, self.episode, minutes, seconds);
    }
}

#[derive(Debug)]
struct AudioBook {
    title: String,
    author: String,
    narrator: String,
    chapter: u32,
    total_chapters: u32,
    duration: u32,
}

impl AudioBook {
    fn new(title: String, author: String, narrator: String, 
           chapter: u32, total_chapters: u32, duration: u32) -> Self {
        AudioBook { title, author, narrator, chapter, total_chapters, duration }
    }
}

impl Playable for AudioBook {
    fn play(&self) {
        println!("📚 Playing audiobook: '{}' by {} (Chapter {}/{})", 
                self.title, self.author, self.chapter, self.total_chapters);
        println!("    Narrated by {}", self.narrator);
    }
    
    fn pause(&self) {
        println!("⏸️  Paused audiobook: '{}' Chapter {}", self.title, self.chapter);
    }
    
    fn stop(&self) {
        println!("⏹️  Stopped audiobook: '{}'", self.title);
    }
    
    fn duration(&self) -> u32 {
        self.duration
    }
    
    fn title(&self) -> &str {
        &self.title
    }
}

impl Seekable for AudioBook {
    fn seek_to(&self, position: u32) {
        let minutes = position / 60;
        let seconds = position % 60;
        println!("📚 Seeking '{}' Chapter {} to {}:{:02}", 
                self.title, self.chapter, minutes, seconds);
    }
}

// Media player that works with any Playable + Seekable item
struct MediaPlayer {
    playlist: Vec<Box<dyn Playable>>,
    current_index: Option<usize>,
}

impl MediaPlayer {
    fn new() -> Self {
        MediaPlayer {
            playlist: Vec::new(),
            current_index: None,
        }
    }
    
    fn add_media(&mut self, media: Box<dyn Playable>) {
        self.playlist.push(media);
        if self.current_index.is_none() {
            self.current_index = Some(0);
        }
    }
    
    fn play_current(&self) {
        if let Some(index) = self.current_index {
            if let Some(media) = self.playlist.get(index) {
                media.play();
            }
        } else {
            println!("No media in playlist");
        }
    }
    
    fn next(&mut self) {
        if let Some(index) = self.current_index {
            if index + 1 < self.playlist.len() {
                self.current_index = Some(index + 1);
                println!("⏭️  Next track");
                self.play_current();
            } else {
                println!("End of playlist");
            }
        }
    }
    
    fn previous(&mut self) {
        if let Some(index) = self.current_index {
            if index > 0 {
                self.current_index = Some(index - 1);
                println!("⏮️  Previous track");
                self.play_current();
            } else {
                println!("Beginning of playlist");
            }
        }
    }
    
    fn show_playlist(&self) {
        println!("\n📋 Playlist:");
        for (i, media) in self.playlist.iter().enumerate() {
            let marker = if Some(i) == self.current_index { "▶️ " } else { "   " };
            print!("{}{}: ", marker, i + 1);
            media.info();
        }
        println!();
    }
}

fn media_player_demo() {
    println!("\n=== Media Player System Demo ===");
    
    let mut player = MediaPlayer::new();
    
    // Add different types of media
    player.add_media(Box::new(Song::new(
        "Bohemian Rhapsody".to_string(),
        "Queen".to_string(),
        355, // 5:55
        "Rock".to_string()
    )));
    
    player.add_media(Box::new(Podcast::new(
        "Rust Programming Podcast".to_string(),
        "Rustacean".to_string(),
        42,
        2700 // 45 minutes
    )));
    
    player.add_media(Box::new(AudioBook::new(
        "The Rust Programming Language".to_string(),
        "Steve Klabnik".to_string(),
        "AI Narrator".to_string(),
        1,
        20,
        1800 // 30 minutes
    )));
    
    player.add_media(Box::new(Song::new(
        "Thunderstruck".to_string(),
        "AC/DC".to_string(),
        292, // 4:52
        "Hard Rock".to_string()
    )));
    
    // Show playlist
    player.show_playlist();
    
    // Play through playlist
    println!("🎵 Starting playback:");
    player.play_current();
    
    println!();
    player.next();
    
    println!();
    player.next();
    
    println!();
    player.next();
    
    println!();
    player.previous();
    
    player.show_playlist();
}

media_player_demo();

---

## 🧪 Active Recall Checkpoint

**Test your understanding without looking back:**

1. What is the orphan rule and why does it exist?
2. How do you define a trait with default implementations?
3. What's the difference between static and dynamic dispatch?
4. How do associated types differ from generic type parameters?
5. When would you use `dyn Trait` vs generic `<T: Trait>`?
6. Can you implement the same trait for a type multiple times?
7. What are associated constants in traits?
8. How do you specify multiple trait bounds?

**Write your answers below:**

**Your Answers:**
1. 
2. 
3. 
4. 
5. 
6. 
7. 
8. 

---

## 🤔 Reflection Prompt

Consider these questions:

1. **How do traits enable polymorphism in Rust without inheritance?**
2. **What are the trade-offs between static and dynamic dispatch?**
3. **How do Rust traits compare to interfaces in other languages?**
4. **When would you choose associated types over generic parameters?**

Write your thoughts below:

**Your Reflections:**

1. 

2. 

3. 

4. 

---

## 🔮 Preview & Connections

### Coming Up Next: Module System & Organization

In our next lesson, you'll learn about:
- Organizing code with modules and crates
- Visibility rules and the `pub` keyword
- Using external crates and dependencies
- Best practices for code organization

### How This Connects
Traits are fundamental to:
- Defining shared behavior across types
- Enabling polymorphism and code reuse
- Working with Rust's standard library
- Building flexible, extensible systems

---

## ✅ Expected Outcomes

**Self-Assessment Checklist** - Can you:

- [ ] Define traits to specify shared behavior?
- [ ] Implement traits for custom and existing types?
- [ ] Use trait bounds to constrain generic parameters?
- [ ] Work with default implementations and associated types?
- [ ] Understand static vs dynamic dispatch trade-offs?
- [ ] Create and use trait objects effectively?
- [ ] Apply the orphan rule correctly?

If you checked all boxes, excellent! You've mastered Rust's trait system.

---

**🎉 Outstanding Progress!** You now understand how to define shared behavior and build flexible, polymorphic systems in Rust!